# **Importing Everything (Necessary Libraries)**

In [0]:
# importing Everything
import torch
from torch.autograd import Variable
import torchvision.models as models
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import datetime
from google.colab import drive

# **Mounting To Google Drive**

In [0]:
drive.mount('/content/drive')
! ls


# **Create The Symbolic Link **

In [0]:
! ls
! ln -s /content/drive/'My Drive'/hello 

# **Checking Symbolic Link Works or Not**

In [0]:
!ls hello

# **CUDA, Models , Loss function and Hyperparameter Optimizer.**

In [0]:
def Calling_Cuda ():
    # Is GPU Available or Na?
    GPU_available = torch.cuda.is_available()
    
    # shows the GPU Type
    GPU_type = torch.cuda.get_device_name(0)
    
    # shows how many GPU are ther
    How_Many_GPU = torch.cuda.device_count()
    
    use_gpu = torch.cuda.is_available()
    if use_gpu:
        print("Using CUDA")
    
    what_device = torch.device(type='cuda')
    
    # Printing info
    print(GPU_available, "\n", GPU_type, "\n", How_Many_GPU, "\n", what_device)
    
    # Gput info
    !nvidia - smi
    
    # Pushing Everything to CUDA.
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # printing CUDA available or cpu
    print(device)
    
    # Downloading pre-trained vgg16 models.
    vgg16_model = models.vgg16(pretrained=True)
    # Pushing vgg16 pre-trained models to CUDA.
    vgg16_model = vgg16_model.to(device)
    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    # Hyperparameter optimizer -- for the Nonlinearity function--
    optimizer = optim.SGD(vgg16_model.parameters(), lr=0.001, momentum=0.9)


# ** Preparing Train & Validation Sets**

In [0]:
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import random

# This is from GitHub...I just tweaked couple things...
# augment will be True
def get_train_valid_loader(augment, valid_size=0.2, shuffle=True, num_workers=0, pin_memory=True):
    
    # Picks a random number/Vector 
    random_seed = random.seed(9001)
  
    # Don't want High or Low numbers - Samples are wide range - Normalize Needed
    # problem normalize doesn't exist -- Exploding gradien.    
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],)
    
    # Define transforms
    # Ensuring that our model is not overfitting or under fitting.
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
    ])
   
    # augment = True : This one always running     
    if augment:
        # Resizing the image, Random Horizontal Flip, and normalize 
        train_transform = transforms.Compose([
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

    # Load the dataset
    # train_dataset -- 50000
    # valid_dataset -- 50000
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform,)
    valid_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=valid_transform,)
    
    # num_train = 50000
    num_train = len(train_dataset)

    # Empty Array/List of 50000
    indices = list(range(num_train))
    
    # Rounded values: (20% * 50000) = 10,000
    split = int(np.floor(valid_size * num_train))
  
    # shuffle = True : This one always running   
    if shuffle:
        np.random.seed(random_seed)
        # LIST OF random NUMBER          
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, sampler=train_sampler,
                                                num_workers=num_workers, pin_memory=pin_memory,)

    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, sampler=valid_sampler,
                                               num_workers=num_workers, pin_memory=pin_memory,)
    
    # returning test set and validation set as tuple
    return (train_loader, valid_loader)
  
# QUICK MATH
# --------------------------------------
# 40,000 Image's For Train --- 63.34%
# 10,000 Image's For Validation --- 20%
# 10,000 Image's For Test --- 16.66%
# --------------------------------------
# 40,000(Images) / 32(batch_size(parallel)) = 1,250 (Each Epoch Batch)
# 1250(Each batch) × 32(batch_size) = 40,000 
# 11% Difference

the_train_loader, the_valid_loader = get_train_valid_loader(True)
# checking --- printing the size 
print(len(the_train_loader))
print(len(the_valid_loader))

Files already downloaded and verified
1250
313


# **Preparing Test Set**

In [0]:
def get_test_loader(batch_size=32, shuffle=True, num_workers=0, pin_memory=True):

    normalize = transforms.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],)

    # define transform
    transform = transforms.Compose([ transforms.Resize(224),transforms.RandomHorizontalFlip(), transforms.ToTensor(), normalize,])
    

    dataset = datasets.CIFAR10(root='./data', train=False,download=True, transform=transform,)

    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=pin_memory,)

    return data_loader

the_test_loader = get_test_loader()
print(len(the_test_loader))

Files already downloaded and verified
313


# **Checking Downloaded File is Available or No?**

In [0]:
# checking is the file is available or na???
! ls

data  drive  hello  sample_data


# **See The Model Visually**

In [0]:
# PRINT THE model to see visually
print(vgg16_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

# **Freezing up all the layers &  Unfreeze Layer 6 --- Classifier (1)**
Train the network with all but the classification layer frozen

In [0]:
def FreezingUp_ONE():
    # Freezing up all the layers
    for param in vgg16_model.features.parameters():
        param.requires_grad = False
    
    # Freezing up all the classifier
    for param in vgg16_model.classifier.parameters():
        param.requires_grad = False

    # Unfreeze Layer 6 --- Classifier
    for param in vgg16_model.classifier[6].parameters():
        param.requires_grad = True

# **Only The Last Two Fully Connected Layers Frozen (2)**

In [0]:
def FreezingUp_TWO():
    # Freezing up all the layers
    for param in vgg16_model.features.parameters():
        param.requires_grad = False
    
    # Freezing up all the classifier
    for param in vgg16_model.classifier.parameters():
        param.requires_grad = False
    
    # Unfreeze Layer 3 --- Classifier
    for param in vgg16_model.classifier[3].parameters():
        param.requires_grad = True

    # Unfreeze Layer 6 --- Classifier
    for param in vgg16_model.classifier[6].parameters():
        param.requires_grad = True


# **Last Convolution Layer &  Last Two Fully Connected Layers Frozen (3)**

In [0]:
def FreezingUp_THREE():
    # Freezing up all the layers
    for param in vgg16_model.features.parameters():
        param.requires_grad = False
    
    # Freezing up all the classifier
    for param in vgg16_model.classifier.parameters():
        param.requires_grad = False
    
    # Unfreeze up all the layers
    for param in vgg16_model.features[28].parameters():
        param.requires_grad = True
        
    # Unfreeze Layer 3 --- Classifier
    for param in vgg16_model.classifier[3].parameters():
        param.requires_grad = True

    # Unfreeze Layer 6 --- Classifier
    for param in vgg16_model.classifier[6].parameters():
        param.requires_grad = True
        

# **Train The Network**

In [0]:
def modal_call(the_data_loader,epoch_size):
    # Start Time
    start_time = datetime.datetime.now()    
    # Finding Accuracy
    best_acc = 0.0
    
    class_correct = list(0. for x in range(10))
    class_total = list(0. for y in range(10))
    
    correct = 0.0
    total = 0.0
    
    for epoch in range(epoch_size):  # Loop Over The Data-set  Multiple Times
        print('Epoch: ', epoch)
        running_loss = 0.0
    
        each_Epoch_time_START = datetime.datetime.now()
        # Iterate Over Data.
        for i, (inputs, labels) in enumerate(the_data_loader):
            # Get the inputs
            inputs = inputs.to(device)
            labels = labels.to(device)
    
            # Zero the parameter gradients
            optimizer.zero_grad()
    
            # Forward + backward + optimize
            outputs = vgg16_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            # Printing Statistics
            running_loss += loss.item()
    
            if i % 250 == 249:  # Print Every 250 (Mini-Batches)
                print('[%d, %5d] Loss: %.3f' % (epoch + 1, i + 1, running_loss / (i + 1)))
    
            # Finding Accuracy the %
            _, preds = torch.max(outputs, 1)
            c = (preds == labels).squeeze()
    
            _, preds2 = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (preds2 == labels).sum().item()
    
        # Printing Statistics
        print("Epoch: ", epoch, " Loss: %.3f" % (running_loss / 1250), " Accuracy: %.2f" % (100 * correct/total), "%")
        each_Epoch_time_DONE = datetime.datetime.now()
        each_Epoch_whole_time = each_Epoch_time_DONE - each_Epoch_time_START
        print("Epoch: ", epoch, "It Takes: ", each_Epoch_whole_time, " Minutes")
    
        # running_loss_two = 0.0
        torch.save(vgg16_model.state_dict(), "hello/model_{0:03d}.pth".format(epoch))
        print("\n")
    
    end_time = datetime.datetime.now()
    the_whole_time = end_time - start_time
    print('Finished Training -- The Whole Time Is: \t', the_whole_time)

# **Test The Network On The Test Data**

In [0]:
def Test_The_Network():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in the_test_loader:
            images, labels = data
    
            # Send it to CUDA
            images = images.to(device)
            labels = labels.to(device)
    
            outputs = vgg16_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# **Confusion Matrix**

In [0]:
def Confusion_Matrix():
    Matrix = [[0 for x in range(10)] for y in range(10)]
    with torch.no_grad():
        for data in the_test_loader:
            images, labels = data
            
            # Send it to CUDA
            images = images.to(device) 
            labels = labels.to(device)
            
            # Model at that time     
            outputs = vgg16_model(images)
            _, predicted = torch.max(outputs, 1)
            
            # 10,016 images tahats why its len()
            for i in range(len(labels)):
                # ground_truth and pred ands up 1 everytime
                ground_truth = labels[i]
                pred = predicted[i]
                Matrix[ground_truth][pred] += 1
                
    return Matrix

# **Classes Perform Accuracy**

In [0]:
def Classes_Perform_Accuracy():
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in the_test_loader:
            images, labels = data
    
            # Get the inputs
            images = images.to(device)
            labels = labels.to(device)
    
            outputs = vgg16_model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
    
    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

# **Result: Training and Testing Network(1) ** 
Train the network with all but the classification layer frozen

In [0]:
def classification_TASK_ONE():
    Calling_Cuda()
    print("\n")
    print("(i) Train the network with all but the classification layer frozen")
    print("________________________________________________________________")
    # Frozen layer
    FreezingUp_ONE()
    print("Training")
    print("---------")
    print(modal_call(the_train_loader,1))
    print("\n")
    print("Testing")
    print("--------")
    print(Test_The_Network())
    print("\n")
    print("Confusion Matrix")
    pritn(Confusion_Matrix())
    print("------------------")
    print("\n")
    print("Classes Perform Accuracy")
    print("---------------------------")
    print(Classes_Perform_Accuracy())
    print("\n DONE \n")


Training
---------
Epoch:  0
[1,   250] Loss: 1.946
[1,   500] Loss: 1.353
[1,   750] Loss: 1.147
[1,  1000] Loss: 1.030
[1,  1250] Loss: 0.955
Epoch:  0  Loss: 0.955  Accuracy: 71.65 %
Epoch:  0 It Takes:  0:11:21.887737  Minutes


Epoch:  1
[2,   250] Loss: 0.640
[2,   500] Loss: 0.638
[2,   750] Loss: 0.637
[2,  1000] Loss: 0.632
[2,  1250] Loss: 0.632
Epoch:  1  Loss: 0.632  Accuracy: 74.81 %
Epoch:  1 It Takes:  0:11:21.351522  Minutes


Epoch:  2
[3,   250] Loss: 0.601
[3,   500] Loss: 0.605
[3,   750] Loss: 0.606
[3,  1000] Loss: 0.607
[3,  1250] Loss: 0.606
Epoch:  2  Loss: 0.606  Accuracy: 76.09 %
Epoch:  2 It Takes:  0:11:22.535176  Minutes


Epoch:  3
[4,   250] Loss: 0.587
[4,   500] Loss: 0.585
[4,   750] Loss: 0.590
[4,  1000] Loss: 0.592
[4,  1250] Loss: 0.594
Epoch:  3  Loss: 0.594  Accuracy: 76.89 %
Epoch:  3 It Takes:  0:11:22.733712  Minutes


Epoch:  4
[5,   250] Loss: 0.584
[5,   500] Loss: 0.589
[5,   750] Loss: 0.582
[5,  1000] Loss: 0.581
[5,  1250] Loss: 0.582


# **Result: Training and Testing Network(2) ** 
Train the network with all but only the last two fully connected layers frozen

In [0]:
def classification_TASK_TWO():
    Calling_Cuda()
    print("\n")
    print("(ii) Train the network with all but only the last two fully connected layers frozen")
    print("________________________________________________________________")
    FreezingUp_TWO()
    print("Training")
    print("---------")
    print(modal_call(the_train_loader,1))
    print("\n")
    print("Testing")
    print("--------")
    print(Test_The_Network())
    print("\n")
    print("Confusion Matrix")
    print(Confusion_Matrix())
    print("------------------")
    print("\n")
    print("Classes Perform Accuracy")
    print("---------------------------")
    print(Classes_Perform_Accuracy())
    print("\n DONE \n")

Training
---------
Epoch:  0
[1,   250] Loss: 1.685
[1,   500] Loss: 1.209
[1,   750] Loss: 1.030
[1,  1000] Loss: 0.931
[1,  1250] Loss: 0.868
Epoch:  0  Loss: 0.868  Accuracy: 73.30 %
Epoch:  0 It Takes:  0:10:38.295604  Minutes


Epoch:  1
[2,   250] Loss: 0.580
[2,   500] Loss: 0.571
[2,   750] Loss: 0.564
[2,  1000] Loss: 0.563
[2,  1250] Loss: 0.557
Epoch:  1  Loss: 0.557  Accuracy: 76.91 %
Epoch:  1 It Takes:  0:10:35.794635  Minutes


Epoch:  2
[3,   250] Loss: 0.514
[3,   500] Loss: 0.513
[3,   750] Loss: 0.516
[3,  1000] Loss: 0.510
[3,  1250] Loss: 0.512
Epoch:  2  Loss: 0.512  Accuracy: 78.65 %
Epoch:  2 It Takes:  0:10:35.205661  Minutes


Epoch:  3
[4,   250] Loss: 0.503
[4,   500] Loss: 0.487
[4,   750] Loss: 0.480
[4,  1000] Loss: 0.480
[4,  1250] Loss: 0.482
Epoch:  3  Loss: 0.482  Accuracy: 79.80 %
Epoch:  3 It Takes:  0:10:35.370932  Minutes


Epoch:  4
[5,   250] Loss: 0.452
[5,   500] Loss: 0.459
[5,   750] Loss: 0.459
[5,  1000] Loss: 0.456
[5,  1250] Loss: 0.457


# **Result: Training and Testing Network(3) ** 
Train the network with all but the last convolution layer and the last two fully connected layers frozen

In [0]:
def classification_TASK_THREE():
    Calling_Cuda()
    print("\n")
    print("(ii) Train the network with all but only the last two fully connected layers frozen")
    print("________________________________________________________________")
    FreezingUp_THREE()
    print("Training")
    print("---------")
    print(modal_call(the_train_loader,1))
    print("\n")
    print("Testing")
    print("--------")
    print(Test_The_Network())
    print("\n")
    print("Confusion Matrix")
    print(Confusion_Matrix())
    print("------------------")
    print("\n")
    print("Classes Perform Accuracy")
    print("---------------------------")
    print(Classes_Perform_Accuracy())
    print("\n DONE \n")

Training
---------
Epoch:  0
[1,   250] Loss: 0.330
[1,   500] Loss: 0.324
[1,   750] Loss: 0.323
[1,  1000] Loss: 0.326
[1,  1250] Loss: 0.325
Epoch:  0  Loss: 0.325  Accuracy: 88.64 %
Epoch:  0 It Takes:  0:11:06.785407  Minutes


Epoch:  1
[2,   250] Loss: 0.265
[2,   500] Loss: 0.273
[2,   750] Loss: 0.275
[2,  1000] Loss: 0.277
[2,  1250] Loss: 0.278
Epoch:  1  Loss: 0.278  Accuracy: 89.39 %
Epoch:  1 It Takes:  0:11:10.065793  Minutes


Epoch:  2
[3,   250] Loss: 0.242
[3,   500] Loss: 0.242
[3,   750] Loss: 0.244
[3,  1000] Loss: 0.245
[3,  1250] Loss: 0.245
Epoch:  2  Loss: 0.245  Accuracy: 90.06 %
Epoch:  2 It Takes:  0:11:07.409041  Minutes


Epoch:  3
[4,   250] Loss: 0.225
[4,   500] Loss: 0.222
[4,   750] Loss: 0.221
[4,  1000] Loss: 0.220
[4,  1250] Loss: 0.220
Epoch:  3  Loss: 0.220  Accuracy: 90.64 %
Epoch:  3 It Takes:  0:11:12.299587  Minutes


Epoch:  4
[5,   250] Loss: 0.211
[5,   500] Loss: 0.203
[5,   750] Loss: 0.202
[5,  1000] Loss: 0.203
[5,  1250] Loss: 0.204


## **Calling All Autobots**

In [0]:
classification_TASK_ONE()
classification_TASK_TWO()
classification_TASK_THREE

# **Show Images**

In [0]:
def plot_images(images, cls_true, cls_pred=None):
    """
    Adapted from https://github.com/Hvass-Labs/TensorFlow-Tutorials/
    """
    fig, axes = plt.subplots(3, 3)

    for i, ax in enumerate(axes.flat):
        # plot img
        ax.imshow(images[i, :, :, :], interpolation='spline16')

        # show true & predicted classes
        cls_true_name = label_names[cls_true[i]]
        if cls_pred is None:
            xlabel = "{0} ({1})".format(cls_true_name, cls_true[i])
        else:
            cls_pred_name = label_names[cls_pred[i]]
            xlabel = "True: {0}\nPred: {1}".format(
                cls_true_name, cls_pred_name
            )
        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])

    plt.show()

# **Checked Model Saved or Not!!!!**

In [0]:
! pwd
! ls hello

/content


# Loading up the **Model** 

In [0]:
model = torch.load('hello/model_005.pth')
print(model)